<a href="https://colab.research.google.com/github/TomAmster/NLP-TAU-2020/blob/master/4_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import optim
from torch.utils.data import DataLoader
import argparse
import os 
import numpy as np
import sys
import time
import json
from tqdm import tqdm
from collections import defaultdict


num_workers = 4

In [0]:
import pandas as pd
import itertools
from google.colab import drive
import matplotlib.pyplot as plt
import csv
import seaborn as sns 
import numpy as np
from  numpy import int32
from collections import Counter, defaultdict
from os import path

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('/content/drive/Shared drives/NLP Seminar')
import model as models

In [0]:
!pip install jsonlines
import jsonlines

# 1.Setting Up Dataset Representation Models

## 1.1 Donwlading GloVE Embedding Matrix

In [0]:
#!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
#!unzip glove*.zip
#!mv "glove.840B.300d.txt" "/content/drive/Shared drives/NLP Seminar/Data"

##1.2 Setting Mimic Dataset Model To Represent Dataset Information
Inputs-
1.   Path to Train/Dev/Test Tokenized Dataset
2.   Dictionaries - ind2w, w2ind, ind2c, c2ind, ind2c_coarse, c2ind_coarse, fine2coarse, desc, desc_plain, train_examples...
3. Fine Labelset size
4. Coarse Labelset Size
5. Maximum Number Of Tokens Per Document

In [0]:
from torch.utils.data import Dataset

In [0]:
class MimicDataset(Dataset):

    def __init__(self, filename, dicts, num_labels_fine, num_labels_coarse, max_len=-1):
    
        print('loading data from ' + filename)
        
        ind2w, w2ind, ind2c, c2ind, ind2c_coarse, c2ind_coarse = dicts['ind2w'], dicts['w2ind'], dicts['ind2c'], dicts['c2ind'], dicts['ind2c_coarse'], dicts['c2ind_coarse']
    
        if filename.endswith('.ndjson'):
            self.notes_labeled = pd.read_json(filename, lines=True, dtype={0:np.int32, 1:np.int32, 3:np.int32})
            self.notes_labeled.columns = ['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LENGTH', 'LABELS']
            self.notes_labeled['TEXT'] = self.notes_labeled['TEXT'].apply(lambda text : [word for sentence in text for word in sentence])
        else:
            raise ValueError('dataset file must have .csv or .ndjson extension')

        self.num_labels_fine = num_labels_fine
        self.num_labels_coarse = num_labels_coarse
        self.max_len = max_len
        
        self.notes_labeled['TEXT_PLAIN'] = self.notes_labeled['TEXT']
        self.notes_labeled['TEXT'] = self.notes_labeled['TEXT'].apply(lambda text : np.array([int(w2ind.get(word) or len(w2ind)+1) for word in text], dtype=np.int32))
        self.notes_labeled['LABELS_COARSE'] = self.notes_labeled['LABELS'].apply(lambda labels : np.array(list(set([idx for idx in ( int(c2ind_coarse.get(str(l).split('.')[0], -1)) for l in labels ) if idx != -1])), dtype=np.int32))
        self.notes_labeled['LABELS'] = self.notes_labeled['LABELS'].apply(lambda labels : np.array([idx for idx in ( int(c2ind.get(str(l), -1)) for l in labels ) if idx != -1], dtype=np.int32))
        
    def __len__(self):
        return len(self.notes_labeled)

    def __getitem__(self, idx):
        item = self.notes_labeled.iloc[[idx]]

        data = item['TEXT'].values[0]

        if self.max_len > -1 and len(data) >= self.max_len:
            data = data[:self.max_len]
        data = torch.LongTensor(data)

        labels_idx_fine = item['LABELS'].values[0]
        target_fine = np.zeros(self.num_labels_fine)
        target_fine[labels_idx_fine] = 1
        target_fine = torch.FloatTensor(target_fine)

        labels_idx_coarse = item['LABELS_COARSE'].values[0]
        target_coarse = np.zeros(self.num_labels_coarse)
        target_coarse[labels_idx_coarse] = 1
        target_coarse = torch.FloatTensor(target_coarse)

        hadm_id = item['HADM_ID'].values[0]
        doc = item['TEXT_PLAIN'].values[0]
        
        return data, target_fine, target_coarse, hadm_id, doc


## 1.3  Setting A Sentence Model To Represent Sentence Information

In [0]:
class MimicDatasetSentences(Dataset):

    def __init__(self, filename, dicts, num_labels_fine, num_labels_coarse, max_len=-1):
    
        print('loading data from ' + filename)

        self.notes_labeled = pd.read_json(filename, lines=True, dtype={0:np.int32, 1:np.int32, 3:np.int32})
        self.notes_labeled.columns = ['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LENGTH', 'LABELS']
                
        self.num_labels_fine = num_labels_fine
        self.num_labels_coarse = num_labels_coarse
        self.max_len = max_len
        ind2w, w2ind, ind2c, c2ind, ind2c_coarse, c2ind_coarse = dicts['ind2w'], dicts['w2ind'], dicts['ind2c'], dicts['c2ind'], dicts['ind2c_coarse'], dicts['c2ind_coarse']

        self.notes_labeled['LABELS_COARSE'] = self.notes_labeled['LABELS'].apply(lambda labels : np.array(list(set([idx for idx in ( int(c2ind_coarse.get(str(l).split('.')[0], -1)) for l in labels ) if idx != -1])), dtype=np.int32))
        self.notes_labeled['LABELS'] = self.notes_labeled['LABELS'].apply(lambda labels : np.array([idx for idx in ( int(c2ind.get(str(l), -1)) for l in labels ) if idx != -1], dtype=np.int32))
       
        #OOV words are given a unique index at end of vocab lookup
        self.notes_labeled['TEXT_PLAIN'] = self.notes_labeled['TEXT'].apply(lambda text : [' '.join(sentence) for sentence in text])
        self.notes_labeled['TEXT'] = self.notes_labeled['TEXT'].apply(lambda text : [np.array([int(w2ind.get(word) or len(w2ind)+1) for word in sentence], dtype=np.int32) for sentence in text])

    def __len__(self):
        return len(self.notes_labeled)

    def __getitem__(self, idx):
        item = self.notes_labeled.iloc[[idx]]
        
        data = item['TEXT'].values[0]
        if self.max_len > -1 and len(data) >= self.max_len:
            data = data[:self.max_len]
        
        labels_idx_fine = item['LABELS'].values[0]
        target_fine = np.zeros(self.num_labels_fine)
        target_fine[labels_idx_fine] = 1
        target_fine = torch.FloatTensor(target_fine)
        
        labels_idx_coarse = item['LABELS_COARSE'].values[0]
        target_coarse = np.zeros(self.num_labels_coarse)
        target_coarse[labels_idx_coarse] = 1
        target_coarse = torch.FloatTensor(target_coarse)
        
        hadm_id = item['HADM_ID'].values[0]
        doc = item['TEXT_PLAIN'].values[0]
        
        return data, target_fine, target_coarse, hadm_id, doc


In [0]:
def collate_sentences(batch):

    data, target_fine, target_coarse, hadm_ids, docs = zip(*batch)

    m_word = max([len(s) for d in data for s in d])
    m_sent = max([len(d) for d in data])

    padded_sents = [np.array([np.pad(s, (0, m_word-len(s)), 'constant') for s in d]) for d in data]
    padded_docs = np.array([np.pad(d, ((0, m_sent-len(d)),(0,0)), 'constant') for d in padded_sents])
    
    docs = [d + [' ']*(m_sent-len(d)) for d in docs]

    data = torch.LongTensor(padded_docs)

    target_fine = torch.stack(target_fine)
    target_coarse = torch.stack(target_coarse)

    return data, target_fine, target_coarse, hadm_ids, docs

# 2.Setting Up Data Loading & Preperation 

In [0]:
import string
import re
from collections import Counter
from collections import defaultdict
from torch.utils.data import Dataset

## 2.1 Setting & Loading Lookup Tables
1.   Voabulary Lookups
2.   ICD Code Lookups
3.   ICD Code Description Lookup
4.   ICD Description One-Hot Vector Lookup



In [0]:
def load_lookups(args, hier=False):
    """
        Inputs:
            args: Input arguments
        Outputs:
            vocab lookups, ICD code lookups, description lookup, description one-hot vector lookup
    """
    print("Loading Lookup Tables - Function load_lookups")
    #get code and description lookups
    if args["Y"] == 'full':
        codes, codes_train_examples, codes_test_examples, codes_rank = load_full_codes(args['data_path'], hier=hier)
    else:
        codes, codes_train_examples, codes_test_examples, codes_rank = load_top_k_codes(args['Y'], hier=hier)


    desc_plain, codes_billable = load_code_descriptions(args["data-dir"])
    codes = codes.intersection(set(desc_plain.keys()))
    codes_valid = {c : True for c in codes}
    
    for c in codes.difference(set(desc_plain.keys())):
        codes_billable[c] = False
        codes_valid[c] = False
    
    codes_coarse = set([str(c).split('.')[0] for c in codes if c != ''])
    ##DELETE
    print("Normal Codes")
    print(sorted(list(codes)))
    print("Coarse Codes")
    print(sorted(list(codes_coarse)))
    ##DELET
    
    if hier:
        codes = codes - codes_coarse
        
    ind2c = {i:str(c) for i,c in enumerate(sorted(codes))}
    ind2c_coarse = {i:str(c) for i,c in enumerate(sorted(codes_coarse))}
    
    desc_plain_sorted = [' '.join(desc_plain[code]) for _, code in sorted(ind2c.items())]
    codes_rank_sorted = {code : codes_rank[code] for _, code in sorted(ind2c.items())}
    codes_train_examples_sorted = {code : codes_train_examples.get(code, 0) for _, code in sorted(ind2c.items())}
    codes_test_examples_sorted = {code : codes_test_examples.get(code, 0) for _, code in sorted(ind2c.items())}
    codes_billable_sorted = {code : codes_billable[code] for _, code in sorted(ind2c.items())}
    codes_valid_sorted = {code : codes_valid[code] for _, code in sorted(ind2c.items())}
    
    #get vocab lookups
    codes_desc = [desc_plain[code] for _, code in sorted(ind2c.items())] if args['embed-desc'] else None
    ind2w, w2ind = load_vocab_dict(args, args['vocab'], codes_desc=codes_desc)
    desc = [torch.LongTensor([int(w2ind.get(word, len(w2ind)+1)) for word in desc_plain[code]]) for _, code in sorted(ind2c.items())]
    desc = torch.nn.utils.rnn.pad_sequence(desc, batch_first=True, padding_value=0)
            
    c2ind = {str(c):i for i,c in ind2c.items()}
    c2ind_coarse = {str(c):i for i,c in ind2c_coarse.items()}
    fine2coarse = np.zeros(len(ind2c))
    for idx, code in ind2c.items():
        idx_coarse = c2ind_coarse[code.split('.')[0]]
        fine2coarse[idx] = idx_coarse

    dicts = {'ind2w': ind2w, 'w2ind': w2ind, 'ind2c': ind2c, 'c2ind': c2ind, 'ind2c_coarse': ind2c_coarse, 'c2ind_coarse': c2ind_coarse,
             'fine2coarse' : fine2coarse, 'desc': desc, 'desc_plain' : desc_plain_sorted, 'train_examples' : codes_train_examples_sorted, 'test_examples' : codes_test_examples_sorted, 
             'rank' : codes_rank_sorted, 'billable' : codes_billable_sorted, 'valid' : codes_valid_sorted}

    return dicts

### 2.1.1 Loading Numerical ICD Codes

In [0]:
def load_full_codes(train_path, hier=False):
    """
        Inputs:
            train_path: path to train dataset
        Outputs:
            code lookup, description lookup
    """
    print("Loading Numerical ICD Codes - Function load_full_codes")
    codes = set()
    codes_train_examples = Counter()
    codes_test_examples = Counter()
    codes_rank = Counter()
    #build code lookups from appropriate datasets
    for split in ['train', 'dev', 'test']:
        if train_path.endswith('.ndjson'):
            with jsonlines.open(train_path.replace('train', split), 'r') as f:
                for row in f:
                    codes.update([c.strip('.') for c in row[4]])
                    if split == 'train':
                        codes_train_examples.update([c.strip('.') for c in row[4]])
                    elif split == 'test':
                        codes_test_examples.update([c.strip('.') for c in row[4]])
                    codes_rank.update([c.strip('.') for c in row[4]])
        else:
            with open(train_path.replace('train', split), 'r') as f:
                lr = csv.reader(f)
                next(lr)
                for row in lr:
                    codes.update([c.strip('.') for c in row[3].split(',')])
                    if split == 'train':
                        codes_train_examples.update([c.strip('.') for c in row[3]])
                    elif split == 'test':
                        codes_test_examples.update([c.strip('.') for c in row[3]])
                    codes_rank.update([c.strip('.') for c in row[3].split(';')])
    codes = set([str(c) for c in codes if c != ''])
    codes_train_examples = {code : count for code, count in codes_train_examples.most_common()}
    codes_test_examples = {code : count for code, count in codes_test_examples.most_common()}
    codes_rank = {code : rank+1 for rank, (code, _) in enumerate(codes_rank.most_common())}
    
    return codes, codes_train_examples, codes_test_examples, codes_rank
    

### 2.1.2 Loading Textual ICD Codes Description Dictionary

Returns Dictionary- Numerical Code: Code Description

In [0]:
def load_code_descriptions(data_dir):
    #load description lookup from the appropriate data files
    print("Loading ICD Codes Description - Function load_code_descriptions")

    desc_dict_plain = defaultdict(str)
    codes_billable = {}
    
    with open(os.path.join(data_dir, 'D_ICD_DIAGNOSES.csv'), 'r') as descfile:
        r = csv.reader(descfile)
        #header
        next(r)
        for row in r:
            code = reformat(row[1], True)
            desc = row[-1]
            desc_dict_plain[code] = [word.lower() for word in re.split('\W+', desc) if word.isalpha()]
            codes_billable[code] = True

    with open(os.path.join(data_dir, 'D_ICD_PROCEDURES.csv'), 'r') as descfile:
        r = csv.reader(descfile)
        #header
        next(r)
        for row in r:
            code = reformat(row[1], False)
            desc = row[-1]
            desc_dict_plain[code] = [word.lower() for word in re.split('\W+', desc) if word.isalpha()]
            codes_billable[code] = True
    
    with open('/content/drive/Shared drives/NLP Seminar/Data/ICD9_descriptions.txt', 'r') as labelfile:
      for i,row in enumerate(labelfile):
        row = row.rstrip().split()
        code = str(row[0])
        if code not in desc_dict_plain.keys():
          desc = ' '.join(row[1:])
          desc_dict_plain[code] = [word.lower() for word in re.split('\W+', desc) if word.isalpha()]
          codes_billable[code] = False
    print("desc_dict_plain = ", desc_dict_plain)
    print("codes_billable = ",codes_billable)
    return desc_dict_plain, codes_billable

### 2.1.3 Format ICD Codes To Proper Structure Including Periods

In [0]:
def reformat(code, is_diag):
    """
        Put a period in the right place because the MIMIC-3 data files exclude them.
        Generally, procedure codes have dots after the first two digits, 
        while diagnosis codes have dots after the first three digits.
    """
    if code == '':
        return code
        
    code = ''.join(code.split('.'))
    if is_diag:
        if code.startswith('E'):
            if len(code) > 4:
                code = code[:4] + '.' + code[4:] if len(code) > 4 else code
        else:
            code = code[:3] + '.' + code[3:] if len(code) > 3 else code
    else:
        code = code[:2] + '.' + code[2:] if len(code) > 2 else code
    return code

### 2.1.4 Load Vocabulary Dictionary Lookups Ind2W, W2Ind
1. Create Ind2W Dictionary
2. Create W2Ind Dictionary

In [0]:
def load_vocab_dict(args, vocab_file, codes_desc=None):
    #reads vocab_file into two lookups (word:ind) and (ind:word)
    print("Loading Voabulary Dictionaries, Ind2W & W2Ind - Function load_vocab_dict")

    vocab = set()
    
    with open(vocab_file, 'r') as vocabfile:
        for i,line in enumerate(vocabfile):
            line = line.rstrip()
            if line != '':
                vocab.add(line.strip())
    if codes_desc is not None:
        desc = set([w for d in codes_desc for w in d])
        vocab = vocab.union(desc)
    ind2w = {i+1:w for i,w in enumerate(sorted(vocab))}
    w2ind = {w:i for i,w in ind2w.items()}

    
    return ind2w, w2ind


## 2.2 Defining Collate Function For Data Loaders Future Use - TOM COME BACK TO THIS

In [0]:
def collate(batch):

    data, target_fine, target_coarse, hadm_ids, docs = zip(*batch)

    data = torch.nn.utils.rnn.pad_sequence(data, batch_first=True, padding_value=0)
    
    target_fine = torch.stack(target_fine)
    target_coarse = torch.stack(target_coarse)

    return data, target_fine, target_coarse, hadm_ids, docs

# 3.Setting Up Training Model & Configuraiton 

## 3.1 Initiating Datasets And Building Model
1.   Loading Lookup Tables
2.   Building The Model



In [0]:
def init(args):
    """
        Load data, build model, create optimizer, create vars to hold metrics, etc.
    """

    #load vocab and other lookups
    print("loading lookups...")

    dicts =load_lookups(args, hier=args["hier"])

    model, optimizer = init_model(args, dicts)
        
    print(model)

    params = args
    
    return args, model, optimizer, params, dicts


## 3.2 Load Embedding Matrix From Embedding File

In [0]:
def load_embeddings(embed_file, ind2w, embed_size, embed_normalize=False):
    print("Loadding Embedding Matrix - Function load_embedding")
    word_embeddings = {}
    vocab_size = len(ind2w)
    
    with open(embed_file) as ef:
        for line in ef:
            line = line.rstrip().split()
            idx = len(line) - embed_size
            word = '_'.join(line[:idx]).lower().strip()
            vec = np.array(line[idx:]).astype(np.float)
            word_embeddings[word] = vec

    W = np.zeros((vocab_size+2, embed_size))
    words_found = 0
    for ind, word in ind2w.items():

        try: 
            W[ind] = word_embeddings[word]
            words_found += 1
        except KeyError:
            W[ind] = np.random.randn(1, embed_size)
        if embed_normalize:
            W[ind] = W[ind] / (np.linalg.norm(W[ind]) + 1e-6)

    W[vocab_size-1] = np.random.randn(1, embed_size)
    
    if embed_normalize:
        W[vocab_size-1] = W[vocab_size-1] / (np.linalg.norm(W[vocab_size-1]) + 1e-6)

    print('vocabulary coverage: {}'.format(words_found/vocab_size))
    
    return W

## 3.3 Initialzing The Neural Network Model

In [0]:
def init_model(args, dicts):
    """
        Use args to initialize the appropriate model
    """
   
    print("Initializing NN Model - Function init_model")

    Y = len(dicts['ind2c'])
    Y_coarse = len(dicts['ind2c_coarse']) if args['hier'] else None
    
    print("loading pretrained embeddings...")            
    word_embeddings_matrix = load_embeddings(args['embed-file'], dicts['ind2w'], args['dims'][0])

        
    vocab_size = len(dicts['ind2w'])
                    
    if args['model'] == "conv_dilated":
        model = models.ConvDilated(Y, args['dims'], args['filter-size'], args['dilation'], word_embeddings_matrix, args['gpu'], vocab_size,
                                    embed_freeze=False, dropout=args['dropout'],
                                    hier=args['hier'], Y_coarse=Y_coarse, fine2coarse=dicts['fine2coarse'],
                                    embed_desc=args['embed-desc'])
    elif args['model'] == "conv_attn":
        model = models.ConvAttnPool(Y, args['dims'], args['filter-size'], word_embeddings_matrix, args['gpu'], vocab_size,
                                    embed_freeze=False, dropout=args['dropout'],
                                    hier=args['hier'], Y_coarse=Y_coarse, fine2coarse=dicts['fine2coarse'],
                                    embed_desc=args['embed-desc'])
    if args['gpu']:
        model.cuda()

    optimizer = optim.Adam(model.parameters(), weight_decay=args['weight-decay'], lr=args['lr'])
    return model, optimizer

## 3.4 Building Trainning Configuration

1. Creating Dataset Object For Train & Dev
2. Perform Training On Epochs

In [0]:
def train_epochs(args, model, optimizer, params, dicts):
    """
        Main loop. does train and test
    """
    print("Setting Up Training Configuration - Function train_epochs")
    metrics_hist_train = defaultdict(list)
    metrics_hist_dev = defaultdict(list)
    metrics_hist_test = defaultdict(list)
             
    num_labels_fine = len(dicts['ind2c'])
    num_labels_coarse = len(dicts['ind2c_coarse'])

    epoch = 0 
    

    dataset_train = MimicDataset(args['data_path'], dicts, num_labels_fine, num_labels_coarse, args['max-len'])
    dataset_dev = MimicDataset(args['data_path'].replace('train', 'dev'), dicts, num_labels_fine, num_labels_coarse, args['max-len'])
    model_dir = os.path.join(args['models-dir'], '_'.join([args['model'], time.strftime('%Y-%m-%d_%H:%M:%S')]))
    os.mkdir(model_dir)

    #tensorboard = Tensorboard(model_dir)
    
    #train for n_epochs unless criterion metric does not improve for [patience] epochs
    for epoch in range(epoch, args['n-epochs']):
   
        losses = train(model, optimizer, args['Y'], epoch, args['batch-size'], args['embed-desc'], dataset_train, args['gpu'], dicts)
        loss = np.mean(losses)
        
        metrics_train = {'loss': loss}

        fold ='dev'

        #evaluate on dev
        with torch.no_grad():
            metrics_dev, _, _, _ = test(model, args['Y'], epoch, dataset_dev, args['batch-size'], args['embed-desc'], fold, args['gpu'], dicts, model_dir)

        for name, val in metrics_train.items():
            #tensorboard.log_scalar('%s_train' % (name), val, epoch)
            metrics_hist_train[name].append(metrics_train[name])
            metrics_hist_train.update({'epochs': epoch+1})
        for name, val in metrics_dev.items():
            #tensorboard.log_scalar('%s_dev' % (name), val, epoch)
            metrics_hist_dev[name].append(metrics_dev[name])

        metrics_hist_all = (metrics_hist_train, metrics_hist_dev, None)

        #save metrics, model, optimizer state, params
        save_everything(args, dicts, metrics_hist_all, model, optimizer, model_dir, params, args['criterion'], evaluate=False, test_only=False)

        if args['criterion'] is not None:
            if early_stop(metrics_hist_dev, args['criterion'], args['patience']):
                #stop training, evaluate on test set and then stop the script
                print('{} hasn\'t improved in {} epochs, early stopping...'.format(args['criterion'], args['patience']))
                break

    fold = 'test'            
    print("\nevaluating on test")
    
    dataset_train = None
    dataset_dev = None
    del dataset_train, dataset_dev
    model_best_sd = torch.load(os.path.join(model_dir, 'model_best_{}.pth'.format(args['criterion'])))
    model.load_state_dict(model_best_sd)
    
    if args['gpu']:
        model.cuda()
    
    dataset_test = MimicDataset(args['data_path'].replace('train', 'test'), dicts, num_labels_fine, num_labels_coarse, args['max-len'])
    
    with torch.no_grad():
        metrics_test, metrics_codes, metrics_inst, hadm_ids = test(model, args['Y'], epoch, dataset_test, args['batch-size'], args['embed-desc'],fold, args['gpu'], dicts, model_dir)
    
    for name, val in metrics_test.items():
        #if not test_only:
        #    tensorboard.log_scalar('%s_test' % (name), val, epoch)
        metrics_hist_test[name].append(metrics_test[name])
    
    metrics_hist_all = (metrics_hist_train, metrics_hist_dev, metrics_hist_test)

    #tensorboard.close()
        
    #save metrics, model, params
    save_everything(args, dicts, metrics_hist_all, model, optimizer, model_dir, params, args['criterion'], metrics_codes=metrics_codes, metrics_inst=metrics_inst, hadm_ids=hadm_ids, evaluate=True, test_only=test_only)

    return epoch+1, metrics_hist_test


## 3.5 Preform Training On A Specific Batch

In [0]:
def train(model, optimizer, Y, epoch, batch_size, embed_desc, dataset, gpu, dicts, shuffle=False):
    """
        Training loop.
        output: losses for each example for this iteration
    """
    print("EPOCH %d" % epoch)
    
    #accumulation_steps = batch_size/8
    #assert batch_size % 8 == 0
    #optimizer.zero_grad()
    #batch_size = 8

    losses = []

    ind2w, w2ind, ind2c, c2ind, desc = dicts['ind2w'], dicts['w2ind'], dicts['ind2c'], dicts['c2ind'], dicts['desc']

    model.train()
    gen = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=collate, pin_memory=True)
        
    desc_data = desc
    if embed_desc and gpu:
        desc_data = desc_data.cuda()
    
    t = tqdm(gen, total=len(gen), ncols=0, file=sys.stdout)
    for batch_idx, tup in enumerate(t):
        data, target, target_coarse, _, _ = tup
        if gpu:
            data, target, target_coarse = data.cuda(), target.cuda(), target_coarse.cuda()
        seq_length = data.size()[1]
        
        optimizer.zero_grad()
        if model.hier:
            _, loss, _ = model(data, target, target_coarse, desc_data=desc_data)
        else:
            _, loss, _ = model(data, target, desc_data=desc_data)
        
        del data, target, target_coarse
        loss.backward()
        losses.append(loss.item())
        del loss
      
        optimizer.step()        
        t.set_postfix(batch_size=batch_size, seq_length=seq_length, loss=np.mean(losses))
    
    return losses

## 3.6 Define Early Stop Configuraiton When A Metric Stops Improving

In [0]:
def early_stop(metrics_hist, criterion, patience):
    
    assert len(metrics_hist[criterion]) > 0
        
    #keep training if criterion results have all been nan so far
    if np.all(np.isnan(metrics_hist[criterion])):
        return False
        
    if criterion == 'loss_dev': 
        return np.nanargmin(metrics_hist[criterion]) > len(metrics_hist[criterion]) - patience
    else:
        return np.nanargmax(metrics_hist[criterion]) < len(metrics_hist[criterion]) - patience

# 4.Test Model Routine

In [0]:
def test(model, Y, epoch, dataset, batch_size, embed_desc, fold, gpu, dicts, model_dir):
    """
        Testing loop.
        Returns metrics
    """

    print('file for evaluation: %s' % fold)

    docs, attention, y, yhat, yhat_raw, hids, losses = [], [], [], [], [], [], []

    y_coarse, yhat_coarse, yhat_coarse_raw = [], [], []

    ind2w, w2ind, ind2c, c2ind, desc = dicts['ind2w'], dicts['w2ind'], dicts['ind2c'], dicts['c2ind'], dicts['desc']

    model.eval()

    gen = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=collate)

    desc_data = desc
    if embed_desc and gpu:
        desc_data = desc_data.cuda()

    t = tqdm(gen, total=len(gen), ncols=0, file=sys.stdout)
    for batch_idx, tup in enumerate(t):
        data, target, target_coarse, hadm_ids, data_text = tup
        
        if gpu:
            data, target, target_coarse = data.cuda(), target.cuda(), target_coarse.cuda()

        model.zero_grad()

        if model.hier:
            output, loss, alpha = model(data, target, target_coarse, desc_data=desc_data)
        else:
            output, loss, alpha = model(data, target, desc_data=desc_data)

        if model.hier:
            output, output_coarse = output
            output_coarse = output_coarse.data.cpu().numpy()
            alpha, alpha_coarse = alpha
        else:
            output_coarse = np.zeros([len(output), len(dicts['ind2c_coarse'])])
            for i, y_hat_raw_ in enumerate(output.data.cpu().numpy()):
                if len(np.nonzero(np.round(y_hat_raw_))) == 0:
                    continue
                codes = [str(dicts['ind2c'][ind]) for ind in np.nonzero(np.round(y_hat_raw_))[0]]
                codes_coarse = set(str(code).split('.')[0] for code in codes)
                codes_coarse_idx = [dicts['c2ind_coarse'][code_coarse] for code_coarse in codes_coarse]
                output_coarse[i, codes_coarse_idx] = 1

        target_coarse_data = target_coarse.data.cpu().numpy()
        y_coarse.append(target_coarse_data)
        yhat_coarse_raw.append(output_coarse)
        yhat_coarse.append(np.round(output_coarse))
        
        losses.append(loss.item())
        target_data = target.data.cpu().numpy()
 
        del data, loss
        
       
        del target
        
        output = output.data.cpu().numpy()
        
        #save predictions, target, hadm ids
        yhat_raw.append(output)
        yhat.append(np.round(output))
        y.append(target_data)
        
        hids.extend(hadm_ids)
        docs.extend(data_text)
        attention.extend(alpha[:,[dicts['c2ind'][c] for c in get_codes()]].cpu())
        
        t.set_postfix(loss=np.mean(losses))
    
    level = ''
    y_coarse = np.concatenate(y_coarse, axis=0)
    yhat_coarse = np.concatenate(yhat_coarse, axis=0)
    yhat_coarse_raw = np.concatenate(yhat_coarse_raw, axis=0)
    metrics_coarse, _, _ = all_metrics(yhat_coarse, y_coarse, yhat_raw=yhat_coarse_raw, level='coarse')
    print_metrics(metrics_coarse, level='coarse')

    y = np.concatenate(y, axis=0)
    yhat = np.concatenate(yhat, axis=0)
    yhat_raw = np.concatenate(yhat_raw, axis=0)
    
    #get metrics
    print("test, all metrics")
    metrics = all_metrics(yhat, y, yhat_raw=yhat_raw, level='fine')
    print_metrics(metrics, level='fine')
    metrics['loss'] = np.mean(losses)
    metrics.update(metrics_coarse)

    #write the predictions
    if fold == 'test':
        write_preds(hids, docs, attention, y, yhat, yhat_raw, model_dir, fold, ind2c, c2ind, dicts['desc_plain'])
    
    return metrics, hids

# 5.MAIN - Set Up Arguments And  Invoke Training Sequence

In [36]:
start = time.time()
num_workers=4
arguments =  {
           "data_path": "/content/drive/Shared drives/NLP Seminar/Data/notes_labeled_train.ndjson",
           "vocab": "/content/drive/Shared drives/NLP Seminar/Data/vocab_words.csv",
           "embed-file":"/content/drive/Shared drives/NLP Seminar/Data/glove.840B.300d.txt" ,
           "Y": "full",
           "model": "conv_dilated",
           "dims": [300,125,125,125],
           "n-epochs" :1,
           "filter-size":3 ,
           # filter size  - size of convolution filter to use. (default: 5)
           "dilation": [1,2,4] ,
           # Dilation - optional specification of dilation (default: 1)"
           "weight-decay": 0 ,
           # Coefficient for penalizing l2 norm of model weights (default: 0)
           "lr":0.0001,
           # lr 0 initial learning rate for Adam optimizer (default=1e-3)
           "batch-size": 8 ,
           #batch-size : size of training batches
           "dropout": [0.2,0.2,0.2] ,
           #dropout - optional specification of dropout (default: 0.5)
           "models-dir": "/content/drive/Shared drives/NLP Seminar/Data/Saved Models" ,
           #models-dir - path to saved models directory"
           "data-dir" : '/content/drive/Shared drives/NLP Seminar/Data',
           #data-dir - path to mimic data directory" 
           "criterion" : "f1_micro_fine" ,
           #criterion - which metric to use for early stopping (default: f1_micro)
           "patience": 10,
           #patience - how many epochs to wait for improved criterion metric before early stopping (default: 3)"
           "gpu": True ,
           #gpu -  optional flag to use GPU if available"           
           "max-len":5200 ,
           #max-len - set maximum number of tokens per document (optional)"
           "hier": False ,
           #hier - hierarchical predictions (default false)"
           "embed-desc": "" ,
            }
args, model, optimizer, params, dicts = init(arguments)
epochs_trained, metrics_hist_test = train_epochs(args, model, optimizer, params, dicts)
elapsed = round(time.time() - start)
m, s = divmod(elapsed, 60)
h, m = divmod(m, 60)
print("TOTAL ELAPSED TIME FOR {} MODEL AND {} EPOCHS: {:d}:{:02d}:{:02d}".format(args['model'], epochs_trained, h, m, s))
print(metrics_hist_test)

loading lookups...
Loading Lookup Tables - Function load_lookups
Loading Numerical ICD Codes - Function load_full_codes
Loading ICD Codes Description - Function load_code_descriptions
desc_dict_plain =  defaultdict(<class 'str'>, {'011.66': ['tuberculous', 'pneumonia', 'any', 'form', 'tubercle', 'bacilli', 'not', 'found', 'by', 'bacteriological', 'or', 'histological', 'examination', 'but', 'tuberculosis', 'confirmed', 'by', 'other', 'methods', 'inoculation', 'of', 'animals'], '011.70': ['tuberculous', 'pneumothorax', 'unspecified'], '011.71': ['tuberculous', 'pneumothorax', 'bacteriological', 'or', 'histological', 'examination', 'not', 'done'], '011.72': ['tuberculous', 'pneumothorax', 'bacteriological', 'or', 'histological', 'examination', 'unknown', 'at', 'present'], '011.73': ['tuberculous', 'pneumothorax', 'tubercle', 'bacilli', 'found', 'in', 'sputum', 'by', 'microscopy'], '011.74': ['tuberculous', 'pneumothorax', 'tubercle', 'bacilli', 'not', 'found', 'in', 'sputum', 'by', 'micro

NameError: ignored

# Save Models and Arguments Auxillary Functions

In [0]:
def get_codes():
    return ['427.31', '584.9', '428.0', '401.9', '518.81',  '787.02', '785.4', '038.12', '501']

In [0]:
def save_metrics(dicts, metrics_hist, model_dir, metrics_codes=None, metrics_inst=None, hadm_ids=None, test_only=False):
    filename = "metrics.json" if not test_only else "metrics_test_only.json"
    with open(os.path.join(model_dir, filename), 'w') as metrics_file:
        #concatenate train, dev and test metrics into one dict
        metrics_hist_train, metrics_hist_dev, metrics_hist_test = metrics_hist
        data = {}
        data.update({"%s_train" % (name):val for (name,val) in metrics_hist_train.items()})
        data.update({"%s_dev" % (name):val for (name,val) in metrics_hist_dev.items()})
        if metrics_hist_test is not None:
            data.update({"%s_test" % (name):val for (name,val) in metrics_hist_test.items()})
        json.dump(data, metrics_file, indent=1)
    if metrics_codes is not None and metrics_inst is not None:
        with open(model_dir + '/metrics_per_code.csv', 'w') as f1, open(model_dir + '/metrics_per_doc.csv', 'w') as f2:
            f1_writer = csv.writer(f1, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
            f1_writer.writerow(['code', 'desc', 'valid', 'billable', 'rank', 'train examples', 'test examples', 'accuracy', 'precision', 'recall', 'f1'])
            for row in zip(dicts['ind2c'].values(), dicts['desc_plain'], dicts['valid'].values(), dicts['billable'].values(), dicts['rank'].values(), dicts['train_examples'].values(), dicts['test_examples'].values(), *metrics_codes):
                f1_writer.writerow(row)
            f2_writer = csv.writer(f2, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
            f2_writer.writerow(['hadm_id', 'accuracy', 'precision', 'recall', 'f1'])
            for row in zip(hadm_ids, *metrics_inst):
                f2_writer.writerow(row)


In [0]:
def save_params_dict(params):
    with open(os.path.join(params["model-dir"], "params.json"), 'w') as params_file:
        json.dump(params, params_file, indent=1)


In [0]:
def write_docs_top_10(model_dir, fold, df, c2ind, desc_plain):

    if not os.path.exists(os.path.join(model_dir, 'qualitative')):
        os.makedirs(os.path.join(model_dir, 'qualitative'))
    docs_dir = 'qualitative'
    
    for i, code in enumerate(get_codes()):
        ind = c2ind[code]
        
        docs_tp = df[df.apply(lambda x: bool(x['target'][ind] == 1) & bool(x['prediction'][ind] == 1), axis=1)]
        #docs_tp.attention = docs_tp['attention'].apply(lambda a : a[ind])
        n_samples = 10 if len(docs_tp)>=10 else len(docs_tp)
        if n_samples > 0:
            docs_tp = docs_tp.sample(n=n_samples)[['id', 'text', 'attention']].reset_index()
            docs_tp['attention'] = docs_tp['attention'].apply(lambda a : int(np.argmax(a[i])))
            out_path = os.path.join(model_dir, docs_dir, code + '_tp.csv')
            docs_tp[['text', 'attention']].apply(lambda row : ' '.join(row['text'][(row['attention']-7 if row['attention']-7 >= 0 else 0):row['attention']+8]), axis=1).to_csv(out_path, sep=' ', index=False, header=[desc_plain[ind]])
            #out_path = model_dir + '/docs_' + code + '_tp_' + fold + '.json'
            #with open(out_path, 'w') as f:
            #    json.dump(json.loads(docs_tp.sample(n=n_samples)[['id', 'text', 'attention']].to_json(orient='records')), f, indent=1)
        
        docs_fn = df[df.apply(lambda x: bool(x['target'][ind] == 1) & bool(x['prediction'][ind] == 0), axis=1)]
        n_samples = 10 if len(docs_fn)>=10 else len(docs_fn)
        if n_samples > 0:
            docs_fn = docs_fn.sample(n=n_samples)[['id', 'text', 'attention']].reset_index()
            docs_fn['attention'] = docs_fn['attention'].apply(lambda a : int(np.argmax(a[i])))
            out_path = os.path.join(model_dir, docs_dir, code + '_fn.csv')
            docs_fn[['text', 'attention']].apply(lambda row : ' '.join(row['text'][(row['attention']-7 if row['attention']-7 >= 0 else 0):row['attention']+8]), axis=1).to_csv(out_path, sep=' ', index=False, header=[desc_plain[ind]])

In [0]:
def write_docs(model_dir, fold, df, lower, upper):
    assert upper >= lower
    
    docs = df[(df['f1']>=lower) & (df['f1']<=upper)]
    n_samples = 5 if len(docs)>=5 else len(docs)
    docs = docs.sample(n=n_samples)
    
    if len(docs) < 1:
        return
            
    i = 0
    for _, doc in docs.iterrows():
        i += 1
        tp = np.nonzero((doc['target'] == 1) & (doc['prediction'] == 1))[0]
        fp = [int(c) for c in (doc['target'] == 0) & (doc['prediction'] == 1) if c]
        fn = [int(c) for c in (doc['target'] == 1) & (doc['prediction'] == 0) if c]
            
        doc_tp = pd.DataFrame()
        doc_tp['id'] = [n for n in range(len(tp))]
        doc_tp['text'] = [doc['text'] for n in range(len(tp))]
        doc_tp['attention'] = [a for a in doc['attention'][tp]]
        doc_tp['label'] = [t for t in tp]
        doc_tp['prediction'] = [p for p in tp]
        doc_tp['posterior'] = [0 for p in tp] 


        out_path = os.path.join(model_dir, 'doc_f1_' + str(int(lower*100)) + '_' + str(int(upper*100)) + '_' + str(i) + '_tp_' + fold + '.json')
        with open(out_path, 'w') as f:
            json.dump(json.loads(doc_tp.to_json(orient='records')), f, indent=1)
            
            
        doc_fp = pd.DataFrame()
        doc_fp['id'] = [n for n in range(len(fp))]
        doc_fp['text'] = [doc['text'] for n in range(len(fp))]
        doc_fp['attention'] = [a for a in doc['attention'][fp,:]]
        doc_fp['label'] = [t for t in fp]
        doc_fp['prediction'] = [p for p in fp]
        doc_fp['posterior'] = [0 for p in fp]
        
        out_path = model_dir + '/doc_f1_' + str(int(lower*100)) + '_' + str(int(upper*100)) + '_' + str(i) + '_fp_' + fold + '.json'
        with open(out_path, 'w') as f:
            json.dump(json.loads(doc_fp.to_json(orient='records')), f, indent=1)
            
        doc_fn = pd.DataFrame()
        doc_fn['id'] = [n for n in range(len(fn))]
        doc_fn['text'] = [doc['text'] for n in range(len(fn))]
        doc_fn['attention'] = [a for a in doc['attention'][fn]]
        doc_fn['label'] = [t for t in fn]
        doc_fn['prediction'] = [p for p in fn]
        doc_fn['posterior'] = [0 for p in fn]
                
        out_path = model_dir + '/doc_f1_' + str(int(lower*100)) + '_' + str(int(upper*100)) + '_' + str(i) + '_fn_' + fold + '.json'
        with open(out_path, 'w') as f:
            json.dump(json.loads(doc_fn.to_json(orient='records')), f, indent=1)


In [0]:
def write_preds(hids, docs, attns, ys, yhats, metrics, fold, ind2c, c2ind, desc_plain):
        
    preds_file = os.path.join(model_dir, 'preds_{}.psv'.format(fold))
    with open(preds_file, 'w') as f:
        w = csv.writer(f, delimiter='|')
        for yhat, hid in zip(yhats, hids):
            codes = sorted([ind2c[ind] for ind in np.nonzero(yhat)[0]])
            if len(codes) == 0:
                w.writerow([hid, ''])
            else:
                w.writerow([hid] + list(codes))
                
    preds_file_target = os.path.join(model_dir, 'preds_{}_target.psv'.format(fold))
    with open(preds_file_target, 'w') as f:
        w = csv.writer(f, delimiter='|')
        for y, hid in zip(ys, hids):
            codes = sorted([ind2c[ind] for ind in np.nonzero(y)[0]])
            if len(codes) == 0:
                w.writerow([hid, ''])
            else:
                w.writerow([hid] + list(codes))
                

    metrics_df = pd.DataFrame(data={'id':hids, 'accuracy':metrics[0], 'precision':metrics[1], 'recall':metrics[2], 'f1':metrics[3]})

    docs_df = pd.DataFrame(data={'id':hids, 'text':docs, 'attention':attns, 'target':list(ys), 'prediction':list(yhats)})
    
    docs_df = docs_df.merge(metrics_df, on='id', how='inner')
    
    write_docs_top_10(fold, docs_df, c2ind, desc_plain)
   

In [0]:
def save_everything(args, dicts, metrics_hist_all, model, optimizer, model_dir, params, criterion, metrics_codes=None, metrics_inst=None, hadm_ids=None, evaluate=False, test_only=False):
    """
        Save metrics, model, params all in model_dir
    """
    save_metrics(dicts, metrics_hist_all, model_dir, metrics_codes=metrics_codes, metrics_inst=metrics_inst, hadm_ids=hadm_ids, test_only=test_only)
    params['model-dir'] = model_dir
    
    if not test_only:
        save_params_dict(params)
    
    if evaluate or test_only:
        print("saved metrics to directory %s\n" % (model_dir))
        return
        
    sd = model.cpu().state_dict()
    torch.save(sd, os.path.join(model_dir, "model_last_epoch.pth"))
    
    sd_opt = optimizer.state_dict()
    sd_opt['epoch'] = metrics_hist_all[0]['epochs']
    torch.save(sd_opt, os.path.join(model_dir, "optim_last_epoch.pth"))
    
    #save the model with the best criterion metric
    if not np.all(np.isnan(metrics_hist_all[1][criterion])):
        if np.nanargmax(metrics_hist_all[1][criterion]) == len(metrics_hist_all[1][criterion]) - 1:
            torch.save(sd, os.path.join(model_dir, "model_best_{}.pth".format(criterion)))
            torch.save(sd_opt, os.path.join(model_dir, "optim_best_{}.pth".format(criterion)))
    if args['gpu']:
        model.cuda()
    print("saved metrics, params, model, optmizer state to directory %s\n" % (model_dir))

# Evaluation Auxillary Functions

In [0]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from tqdm import tqdm

In [0]:
def all_metrics(yhat, y, yhat_raw=None, level=None):
    """
        Inputs:
            yhat: binary predictions matrix 
            y: binary  truth matrix
        Outputs:
            dict holding relevant metrics
    """
    
    names = ["jac", "prec", "rec", "f1"]

    #micro
    ymic = y.ravel()
    yhatmic = yhat.ravel()
    micro = all_micro(yhatmic, ymic)

    metrics = {}
    metrics.update({names[i] + "_micro" : micro[i] for i in range(len(micro))})

    #AUC and @k
    #allow k to be passed as int or list

    roc_auc, pr_auc = auc_metrics(yhat_raw, y, ymic)
    metrics.update(roc_auc)
    metrics.update(pr_auc)
       
    return metrics


In [0]:
def all_micro(yhatmic, ymic):
    return micro_jaccard(yhatmic, ymic), micro_precision(yhatmic, ymic), micro_recall(yhatmic, ymic), micro_f1(yhatmic, ymic)

In [0]:
def micro_f1(yhatmic, ymic):
    prec = micro_precision(yhatmic, ymic)
    rec = micro_recall(yhatmic, ymic)
    if prec + rec == 0:
        f1 = 0.
    else:
        f1 = 2*(prec*rec)/(prec+rec)
    return f1

def auc_metrics(yhat_raw, y, ymic):
    if yhat_raw.shape[0] <= 1:
        return

    roc_aucs = []
    roc_auc = {}
    for i in range(y.shape[1]):
        fpr, tpr, _ = roc_curve(y[:,i], yhat_raw[:,i])
        roc_aucs.append(auc(fpr, tpr))
    roc_auc['roc_auc_macro'] = np.nanmean(roc_aucs)

    yhatmic = yhat_raw.ravel()
    fpr, tpr, _ = roc_curve(ymic, yhatmic) 
    roc_auc["roc_auc_micro"] = auc(fpr, tpr)
    
    pr_aucs = []
    pr_auc = {}
    for i in range(y.shape[1]):
        prec, rec, _ = precision_recall_curve(y[:,i], yhat_raw[:,i])
        pr_aucs.append(auc(rec, prec))
    pr_auc['pr_auc_macro'] = np.nanmean(pr_aucs)
    
    prec, rec, _ = precision_recall_curve(ymic, yhatmic)
    pr_auc['pr_auc_micro'] = auc(rec, prec)
    
    return roc_auc, pr_auc